<a href="https://colab.research.google.com/github/smaliyu/AfriNLP/blob/main/hate_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Necessary imports
try:
  import wandb
except:
  !pip install -q wandb
  import wandb
import pandas as pd
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import (XLMRobertaTokenizer, XLMRobertaForSequenceClassification,
                          BertTokenizer, BertForSequenceClassification,
                          AutoTokenizer, AutoModelForSequenceClassification)
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
from tqdm.auto import tqdm
import os


In [2]:
wandb.login()

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

  ········································


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [3]:
# Define hyperparameters in a config dictionary
config = dict(
    epochs=3,
    batch_size=16,
    learning_rate=5e-5,
    dataset="Igbo Tweets",
    architecture="Transformer-based",
    num_labels=None,
)



In [4]:
# Define LabelEncoder
label_encoder = LabelEncoder()

# Load and preprocess datasets
def load_and_preprocess_data(language):
    train_data = pd.read_csv(f'https://raw.githubusercontent.com/smaliyu/AfriNLP/main/datasets/{language}/{language}_train.csv')
    test_data = pd.read_csv(f'https://raw.githubusercontent.com/smaliyu/AfriNLP/main/datasets/{language}/{language}_test.csv')
    train_texts = train_data['tweet'].tolist()
    train_labels = label_encoder.fit_transform(train_data['label'].tolist())
    test_texts = test_data['tweet'].tolist()
    test_labels = label_encoder.transform(test_data['label'].tolist())
    return train_texts, train_labels, test_texts, test_labels

# Tokenization function
def tokenize(texts, tokenizer, max_length=128):
    return tokenizer(texts, add_special_tokens=True, max_length=max_length,
                     padding='max_length', return_attention_mask=True, truncation=True)

class TextDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)



# Training Loop
def train(model, train_loader, optimizer, device, num_epochs):
    model.train()
    for epoch in tqdm(range(num_epochs), desc="Epochs"):
        total_loss = 0
        for batch in train_loader:
            inputs = {k: v.to(device) for k, v in batch.items() if k != 'labels'}
            labels = batch['labels'].to(device)
            optimizer.zero_grad()
            outputs = model(**inputs, labels=labels)
            loss = outputs.loss
            total_loss += loss.item()
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
            optimizer.step()

        avg_train_loss = total_loss / len(train_loader)
        wandb.log({"epoch": epoch, "train_loss": avg_train_loss})

# Evaluation Loop
def evaluate(model, test_loader, device):
    model.eval()
    predictions = []
    with torch.no_grad():
        for batch in test_loader:
            inputs = {k: v.to(device) for k, v in batch.items() if k != 'labels'}
            outputs = model(**inputs)
            logits = outputs.logits
            preds = torch.argmax(logits, dim=1)
            predictions.extend(preds.cpu().numpy())
    return predictions


In [5]:
# Initialize WandB with the config
wandb.init(project="hate-models", config=config)
config = wandb.config

    # Models and languages
models = ["xlm-roberta-base", "bert-base-multilingual-cased","morit/XLM-T-full-xnli","Davlan/naija-twitter-sentiment-afriberta-large"]
languages = ["igbo"]

# Loop over models and languages
for model_name in models:
     # Update tokenizer in config based on current model
    wandb.config.update({"tokenizer": model_name}, allow_val_change=True)

    for language in languages:
            print(f"Training {model_name} on {language} dataset")
            train_texts, train_labels, test_texts, test_labels = load_and_preprocess_data(language)

            # Update number of labels in config based on current dataset
            wandb.config.update({"num_labels": len(np.unique(train_labels))}, allow_val_change=True)

            # Tokenizer and Model
            tokenizer = AutoTokenizer.from_pretrained(model_name)
            model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=wandb.config.num_labels)

             # Tokenize data
            train_encodings = tokenize(train_texts, tokenizer)
            test_encodings = tokenize(test_texts, tokenizer)

            # Create Datasets and DataLoaders
            train_dataset = TextDataset(train_encodings, train_labels)
            test_dataset = TextDataset(test_encodings, test_labels)
            train_loader = DataLoader(train_dataset, batch_size=config.batch_size, shuffle=True)
            test_loader = DataLoader(test_dataset, batch_size=config.batch_size)

            # Define optimizer
            optimizer = torch.optim.AdamW(model.parameters(), lr=config.learning_rate)

            # Device setup
            device = 'cuda' if torch.cuda.is_available() else 'cpu'
            model.to(device)

            # Add WandB watch
            wandb.watch(model, log='all', log_freq=10)

            # Train and Evaluate
            train(model, train_loader, optimizer, device, config.epochs)
            predictions = evaluate(model, test_loader, device)
            test_accuracy = accuracy_score(test_labels, predictions)
            wandb.log({"test_accuracy": test_accuracy})

            print(f"Test Accuracy for {model_name} on {language}: {test_accuracy}")



# Finish WandB run
wandb.finish()



wandb: Currently logged in as: lukmanaj. Use `wandb login --relogin` to force relogin


Training xlm-roberta-base on igbo dataset


config.json:   0%|          | 0.00/615 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.10M [00:00<?, ?B/s]

...

config.json:   0%|          | 0.00/1.10k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/503M [00:00<?, ?B/s]

Epochs:   0%|          | 0/3 [00:00<?, ?it/s]

Test Accuracy for Davlan/naija-twitter-sentiment-afriberta-large on igbo: 0.8823529411764706


epoch,▁▅█▁▅█▁▅█▁▅█
test_accuracy,▁▇██
train_loss,███▇▄▃▆▃▂▅▂▁
epoch,2
test_accuracy,0.88235
train_loss,0.07718
